<a href="https://colab.research.google.com/github/Chantynp/EggsAI/blob/main/Maiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

datos = pd.read_csv('train.csv')
datos['solar_alignment'] = datos['solar_alignment'].map({True: 1, False: 0})
datos['sacrifice_type'] = np.where(
    datos['ritual_intensity'] == 0,
    'sin_sacrificio',
    datos['sacrifice_type']
)

datos = pd.get_dummies(datos, columns=['region', 'sacrifice_type'])

x_entrenamiento = datos.drop(columns=['id', 'corn_yield'])
y_entrenamiento = datos['corn_yield']

x_entrenamiento, x_validacion, y_entrenamiento, y_validacion = train_test_split(
    x_entrenamiento,
    y_entrenamiento,
    test_size=0.2,
    random_state=123,
    shuffle=True
)

modelo = xgb.XGBRegressor(
    objective='reg:squarederror',
    tree_method='hist',
    missing=np.nan,
    n_estimators=1500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0.1,
    reg_alpha=0.5,
    reg_lambda=0.8,
    random_state=42,
    n_jobs=-1
)

modelo.fit(
    x_entrenamiento,
    y_entrenamiento,
)

predicciones = modelo.predict(x_validacion)
rmse = np.sqrt(mean_squared_error(y_validacion, predicciones))
mae = mean_absolute_error(y_validacion, predicciones)
r2 = r2_score(y_validacion, predicciones)

#Impresión del Rendimiento
#print(f"RMSE: {rmse:.2f} kg/ha")
#print(f"MAE: {mae:.2f} kg/ha")
#print(f"R²: {r2:.4f}")

modelo.save_model('modelo.json')

#Prueba
datos_prueba = pd.read_csv('test.csv')
datos_prueba['solar_alignment'] = datos_prueba['solar_alignment'].map({True: 1, False: 0})
datos_prueba['sacrifice_type'] = np.where(
    datos_prueba['ritual_intensity'] == 0,
    'sin_sacrificio',
    datos_prueba['sacrifice_type']
)
datos_prueba = pd.get_dummies(datos_prueba, columns=['region', 'sacrifice_type'])

x_prueba = datos_prueba[x_entrenamiento.columns]
predicciones = modelo.predict(x_prueba)

resultados = pd.DataFrame({
    'id': datos_prueba['id'],
    'corn_yield': predicciones
})

resultados.to_csv('predicciones.csv', index=False)